### A single script utilizing the Preprocessor, LOOCV_error hyperparameter training, and Prediction methods of the GPyS
#### Note that each of the LOOCV_error and prediction methods has their explicit script which have been directly called in this notebook

#### Last modified: July 17th, 2023.

In [1]:
from GPyS_preprocessor import Preprocessor
from GPyS_LOOCV_error import LOOCV
from GPyS_prediction_script import GPyS_Script


from scipy.optimize import minimize

#### Initializing data for example G_12 (the associated physics have been pre-computed - see example_G12.py)


In [2]:
# concatenated orthonormal basis
X = [[0.809017, 0.1045284, -0.6691306,
          -1.0, -0.6691306, 0.1045284,
          0.809017], [0.5877852, 0.9945219,
                      0.7431448, -0.0, -0.7431448,
                      -0.9945219, -0.5877852]]
# sample points
sample = [[0.628319], [1.466077], [2.303835], [3.141593],
              [3.979351], [4.817109], [5.654867]]
# target points
target = [[6.157522]]
# default length scale (precomputed)  - to be changed to optimal length_scale upon hyperparamter training which will then be used for prediction
length_scale = [10.0530966]


#### compute preprocessing quantities

In [3]:
Preprocessor.setup(X)
XtX = Preprocessor.get_XX_cross_product()
VtX = Preprocessor.get_Vt_X()
print("XtX:\n", XtX)
print("VtX:\n", VtX)

XtX:
 [[ 0.99999995  0.66913051 -0.10452852 -0.809017   -0.97814755 -0.5
   0.30901706]
 [ 0.66913051  1.          0.66913063 -0.1045284  -0.80901693 -0.97814762
  -0.5       ]
 [-0.10452852  0.66913063  0.99999995  0.6691306  -0.10452843 -0.80901693
  -0.97814755]
 [-0.809017   -0.1045284   0.6691306   1.          0.6691306  -0.1045284
  -0.809017  ]
 [-0.97814755 -0.80901693 -0.10452843  0.6691306   0.99999995  0.66913063
  -0.10452852]
 [-0.5        -0.97814762 -0.80901693 -0.1045284   0.66913063  1.
   0.66913051]
 [ 0.30901706 -0.5        -0.97814755 -0.809017   -0.10452852  0.66913051
   0.99999995]]
VtX:
 [[ 5.87785200e-01  9.94521900e-01  7.43144800e-01  5.39177249e-17
  -7.43144800e-01 -9.94521900e-01 -5.87785200e-01]
 [-8.09017000e-01 -1.04528400e-01  6.69130600e-01  1.00000000e+00
   6.69130600e-01 -1.04528400e-01 -8.09017000e-01]]


#### hyperparameter training (get optimal length scale from LOOCV method)

In [4]:
length = 10.0530966 # default length scale precomputed for the example_G12 (see set_length_scale function in the G12 class in example_G12.py)
LOO = LOOCV(X, sample=sample, beta=[length])
# compute bounds for objective function optimization (discretion of user)
lenUpper = length * 2
lenLower = length * 0.5
# perform optimization by minimizing objective function
ret = minimize(LOO.hSSDist, lenLower, method='Nelder-Mead', bounds=[(lenLower, lenUpper)], tol=1e-6)
print(ret.x)

[objective: 6.025790385934932]; parameter: [5.0265483]
[objective: 6.035124702297238]; parameter: [5.27787572]
[objective: 6.025790385934932]; parameter: [5.0265483]
[objective: 6.025790385934932]; parameter: [5.0265483]
[5.0265483]


#### compute predictions by calling the GPyS script class methods (see GPyS script in git repository for more details)

In [5]:
length_scale = [5.0265483]

V_circ, sigma2, eps2  = GPyS_Script.principal(
                                        X,
                                        sample,
                                        target,
                                        length_scale,
                                        thetanew_index=0)
print("Vcirc:\n", V_circ, "sigma2:\n,", sigma2, "eps2:\n", eps2)

{
    "X": [
        [
            0.809017,
            0.1045284,
            -0.6691306,
            -1.0,
            -0.6691306,
            0.1045284,
            0.809017
        ],
        [
            0.5877852,
            0.9945219,
            0.7431448,
            -0.0,
            -0.7431448,
            -0.9945219,
            -0.5877852
        ]
    ],
    "i": 0,
    "thetanew": [
        [
            6.157522
        ]
    ],
    "thetanew[i]": [
        6.157522
    ],
    "theta": [
        [
            0.628319
        ],
        [
            1.466077
        ],
        [
            2.303835
        ],
        [
            3.141593
        ],
        [
            3.979351
        ],
        [
            4.817109
        ],
        [
            5.654867
        ]
    ],
    "lengthscale": [
        5.0265483
    ]
}

target_point: [6.157522]

Vcirc:
 [[-0.23472462 -0.97206191]
 [-0.97206191  0.23472462]] sigma2:
, [1.71293204e-05 6.33997545e-03] eps2:
 4.